まず、月の標高データをダウンロードして、ローカルフォルダに保存します。

月の標高データはNASAが観測したデータを利用しやすいように加工して、誰でも利用できるように公開しています。
ここでは、[Moon LRO LOLA DEM 118m](https://astrogeology.usgs.gov/search/map/moon_lro_lola_dem_118m) を利用しました。
このデータはNASAの月探査機ルナー・リコネッサンス・オービター（LRO）搭載の月レーザー高度計（LOLA）のデータに基づいたデジタル標高モデル（DEM）です。
このレーザー高度計は高性能で、真下を測るときには1m以下、水平でも20m以下の誤差で測ることができます。

このデータは月を半径1737.4kmの球体として、周りを包むように置いた円筒に投影して開いた平面のピクセルデータになっています。
1ピクセルが118m(赤道付近)の細かさで標高モデルが作られて、観測されていない部分のデータは周りのデータから補完されています。
全球のデータがもれなく均質にそろっている使いやすいデータです。

ただし、データの量も8GBと、少し大きめなので、ダウンロードには時間がかかり、データ処理をするPCにも相応の量のメモリが必要となります。

進行状況を見ながらダウンロードできるユーティリティ関数を用意しましたので、これを利用してダウンロードしてください。

In [ ]:

import requests
from tqdm import tqdm

def download_with_progress(url, filename):
    """Download file with progress bar"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

月面の標高データのURLと、それを保存するファイル名を指定して、ダウンロードを行います。
(すでにローカルフォルダにダウンロードしたデータがあれば、ダウンロードしません。)

In [ ]:
import os

lunar_dem_url = "https://planetarymaps.usgs.gov/mosaic/Lunar_LRO_LOLA_Global_LDEM_118m_Mar2014.tif"
lunar_dem_file = os.path.join(os.getcwd(), 'data', os.path.basename(lunar_dem_url))
lunar_dem_file

# Download DEM file
if os.path.exists(lunar_dem_file):
    print("Data already downloaded.")
else:
    print("Downloading data...")
    download_with_progress(lunar_dem_url, lunar_dem_file)

[Rasterio](https://github.com/rasterio/rasterio) を使用して月の標高モデルファイルを読み込みます。

Rasterio は、地形のラスタデータを読み書きするためのPythonパッケージです。ここで使う月面DEMのGeoTiffデータを読み込んで、そこに含まれるメタデータなどを利用できます。

In [ ]:
import rasterio
import pprint

lunar_dem_dataset = rasterio.open(lunar_dem_file)

# Display metadata
pprint.pp(lunar_dem_dataset.meta)

読み込んだ月面DEMのデータを3Dモデルへ変換します。

月面DEMは標準半径の球面からの差分の値で記録されているので、基準の球面に凹凸をつけることで月面の地形を表現できます。

そのためにまず、3Dモデル処理パッケージの PyVista で月の平均半径（1,737,400メートル）の球（Sphere）のメッシュモデルを作成します。
次に、その球形のメッシュを構成する各点を月面DEMの標高データにしたがって移動してから、出力するモデルの大きさに縮小して新しいメッシュをつくります。

ここで、月面DEMの標高データをそのままの尺度で利用してしまうと、小さな立体モデルをつくったときにクレーターなどの盛り上がりが低すぎてよく見えません。
そこで、メッシュを変形させるときに変形の度合いを強くすることで、地形を強調できるようにしています。

In [ ]:
import numpy as np
import math
import pyvista as pv
from rasterio.transform import rowcol

def map_data_on_sphere(data, transform, nodata, radius, lon_res, lat_res, data_scale, model_radius):
    
    # 与えられた解像度で球体を作成
    sphere = pv.Sphere(radius=radius, theta_resolution=lon_res, phi_resolution=lat_res)

    # 出力するモデルのスケールを計算
    model_scale = model_radius / radius
    
    # 球体の点からデータを取得し、データに従って出力モデルの点を変更
    points = sphere.points.copy()
    new_points = points.copy()
    
    for i, pt in enumerate(points):
        x, y, z = pt
        r = np.linalg.norm(pt) # 球体の中心(原点)からの距離を計算
        if r == 0:
            continue

        # 球体の表面の点に対応する地理座標(緯度、経度)を計算
        lat_deg = math.degrees(math.asin(z / r))
        lon_deg = math.degrees(math.atan2(y, x))

        # 地理座標(緯度、経度)[度]を投影座標系(x, y)[m]に変換
        # 単純な円筒投影を仮定している
        x_proj = radius * math.radians(lon_deg)
        y_proj = radius * math.radians(lat_deg)
        
        # 球体の表面の点に対応するデータの値を取得
        try:
            row, col = rowcol(transform, x_proj, y_proj)
            elev = data[row, col]
            if elev == nodata:
                elev = 0
        except Exception:
            elev = 0
        
        # データに従って出力モデルの点を計算する
        new_r = r + elev * data_scale # 元の半径にデータの値を加える
        factor = new_r / r # 元の半径と変異した半径との比率
        new_points[i] = pt * factor * model_scale # 元の球体の点の座標と変異率で計算した出力モデルの点
    
    # 出力モデルの点を更新
    sphere.points = new_points
    return sphere


関数 map_data_on_sphere() の動作内容を解説するとこうなります。

- 関数は引数として、標高に換算するデータセット、経度・緯度の解像度（メッシュの細かさを決める）、および標高の効果を調整するスケール係数を受け取ります。

- PyVista により球（Sphere）メッシュを作成します。ここで、theta_resolution と phi_resolution によって緯度・経度方向の細分化を行います。

- 鏡面上の各点（球の各頂点）は、まずデカルト座標 (x, y, z) から球面座標へ変換され、ここから緯度（lat_deg）と経度（lon_deg）を求めます。この際、asin と atan2 を用いて度に変換します。

- 次に、簡易的な円筒射影に基づき、緯度・経度（度単位）を投影座標（メートル単位）に変換します。具体的には、x_proj = radius * radians(lon_deg)、y_proj = radius * radians(lat_deg) として計算されます。

- 得られた投影座標から、Rasterio の rowcol 関数を用いて、DEMの対応する画素（行・列）を取得します。もし読み取られた画素が nodata に該当する場合は、高度（elev）を 0 とします。

- 取得した標高値と引数 data_scale により、各点の半径を調整します。具体的には、新たな半径 new_r = 元の半径 + (elev * data_scale) と計算し、元の点に対してスケールファクター（new_r / r）を掛けることで、点の位置を変更します。

- 最後に、全ての頂点の位置を更新した球メッシュを返します。この球メッシュは、与えられたデータ(標高)に応じて変形（盛り上がったり凹んだり）しているため、地形を表現した3Dモデルとなります。

この関数を使って、3Dプリントに適した解像度や地形の強調度合いに調整して3Dモデルをつくります。

In [ ]:
longitude_resolution = 360 * 4 # 経度方向の解像度
latitude_resolution = 180 * 4  # 緯度方向の解像度
data_scale = 4.0  # DEMの効果を誇張するために調整
model_radius = 50 # 出力モデルの半径[mm]

# 球体にデータをマッピングする
lunar_dem_sphere = map_data_on_sphere(
    data=lunar_dem_dataset.read(1),
    transform=lunar_dem_dataset.transform,
    nodata=lunar_dem_dataset.nodata,
    radius=1737400, # 月の平均半径[mm]
    lon_res=longitude_resolution,
    lat_res=latitude_resolution,
    data_scale=data_scale,
    model_radius=model_radius)


できた3Dモデルを表示してみます。

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(lunar_dem_sphere, show_edges=False, color="white")
plotter.add_title("Lunar DEM\nres={} x {}, scale={}".format(longitude_resolution, latitude_resolution, data_scale))
plotter.show()

```
longitude_resolution = 360 * 4
latitude_resolution = 180 * 4
```

この解像度で球体を作ると、点の数が100万個ほどになります。これは10cmくらいの3Dプリントとして出力するには多すぎます。

元の球体のメッシュは均一な大きさで作られているので、平坦な地形に見られる不必要に細かいメッシュを間引きします。

PyVistaには間引き方法が2種類用意されているので、それぞれの結果を3D表示して比較して、どれを利用するかを決めましょう。

元の月の表面データの値

In [ ]:
lunar_dem_sphere

元のメッシュデータを [pyvista.PolyDataFilters.decimate()](https://pyvista.github.io/pyvista-docs-dev-ja/api/core/_autosummary/pyvista.PolyDataFilters.decimate.html#pyvista.PolyDataFilters.decimate) を使って間引きします。

In [ ]:
target_reduction = 0.90
print(f"Reducing {target_reduction * 100.0} percent out of the original mesh")
decimated = lunar_dem_sphere.decimate(target_reduction)
decimated

元のメッシュデータを [pyvista.PolyDataFilters.decimate_pro()](https://pyvista.github.io/pyvista-docs-dev-ja/api/core/_autosummary/pyvista.PolyDataFilters.decimate_pro.html#pyvista.PolyDataFilters.decimate_pro) を使って間引きします。

In [ ]:
target_reduction = 0.90
print(f"Reducing {target_reduction * 100.0} percent out of the original mesh")
pro_decimated = lunar_dem_sphere.decimate_pro(target_reduction, preserve_topology=True)
pro_decimated

| | N Cells | N Points |
| ---- | ----: | ----: |
| decimated | 206784 | 103445 |
| pro_decimated | 269524 | 134764 |

数値を見ると、両方とも同様に間引きされています。

間引きした結果を3D表示して表面を見てみましょう。

In [ ]:
plotter = pv.Plotter(shape=(1, 3))
plotter.add_mesh(lunar_dem_sphere, show_edges=False, color="white")
plotter.add_title("Original\nCells={}\nPoints={}".format(lunar_dem_sphere.n_cells, lunar_dem_sphere.n_points))
plotter.subplot(0, 1)
plotter.add_mesh(decimated, show_edges=False, color="white")
plotter.add_title("decinate()\nCells={}\nPoints={}".format(decimated.n_cells, decimated.n_points))
plotter.subplot(0, 2)
plotter.add_mesh(pro_decimated, show_edges=False, color="white")
plotter.add_title("decinated_pro()\nCells={}\nPoints={}".format(pro_decimated.n_cells, pro_decimated.n_points))
plotter.show()

3D表示をして表面を拡大してみると、decimated の方がクレーターの形が残っているように見えます。

3Dプリントのために、厚みを指定して殻をつくります。

In [ ]:
def create_shell_from_mesh(mesh, thickness):
    """
    Creates a shell from a mesh by moving points inward by the specified thickness.
    
    Args:
        mesh (pv.PolyData): Input mesh
        thickness (float): Shell thickness
    
    Returns:
        pv.PolyData: Shell mesh combining outer and inner surfaces
    """
    # Make copy of original mesh for inner surface
    inner_mesh = mesh.copy()
    
    # Get points of inner mesh
    points = inner_mesh.points
    
    # For each point, move it toward origin by thickness amount
    for i in range(len(points)):
        point = points[i]
        # Calculate vector from origin to point
        vector = point 
        # Get magnitude of vector
        magnitude = np.linalg.norm(vector)
        if magnitude > 0:
            # Create unit vector
            unit_vector = vector / magnitude
            # Move point inward by thickness along unit vector
            new_point = point - (unit_vector * thickness)
            points[i] = new_point
            
    # Update inner mesh points
    inner_mesh.points = points
    
    # Flip normals of inner mesh
    inner_mesh.flip_normals()
    
    # Combine outer and inner meshes
    shell = mesh.merge(inner_mesh)
    
    return shell

In [ ]:
# Create shell from a mesh
thickness = 2.0 # mm
lunar_dem_shell = create_shell_from_mesh(decimated, thickness)

# Visualize
plotter = pv.Plotter()
plotter.add_mesh(lunar_dem_shell, show_edges=False, color="white", opacity=0.5)
plotter.add_title(f"Lunar Shell\nThickness={thickness}mm")
plotter.show()

STLファイルとして保存します。

In [ ]:
lunar_dem_shell.save("model/lunar_surface_shell.stl")